In [1]:
import os
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from app_diagnosis import get_diadnosis_records
from file_viewer import FileViewer
from models.contours import Contours
from models.diagnosis import Diagnosis
from models.eye_examination import EyeExamination
from models.eye_image import EyeImage
from models.patient import Patient
from models.retina import Retina


class EditRegistrosVentana(tk.Toplevel):
    def __init__(self, parent, records, callback):
        super().__init__(parent)
        self.title("Editar Registro")
        self.records = records
        self.callback = callback
        self.init_ui()

    def init_ui(self):
        self.record_to_edit = None

        # 1. Desplegar los registros existentes
        ttk.Label(self, text="Registros Existentes:").pack(pady=5)
        self.records_list = tk.Listbox(self, width=80, height=10)
        for record in self.records:
            paciente_info = f"Edad: {record.patient.age}, Género: {'M' if record.patient.gender == 0 else 'F'}"
            diagnostico_str = ""
            if record.diagnosis == 0:
                diagnostico_str = "Sano"
            elif record.diagnosis == 1:
                diagnostico_str = "Glaucoma"
            elif record.diagnosis == 2:
                diagnostico_str = "Sospechoso"
            self.records_list.insert(tk.END, f"ID: {record.diagnosis_id}, Diagnóstico: {diagnostico_str}, Paciente: {paciente_info}")
        self.records_list.pack(pady=5)

        # 2. Solicitar Diagnosis_ID a modificar
        ttk.Label(self, text="Ingrese el ID del diagnóstico a modificar:").pack(pady=5)
        self.diagnosis_id_entry = ttk.Entry(self)
        self.diagnosis_id_entry.pack(pady=5)

        ttk.Button(self, text="Seleccionar Registro", command=self.seleccionar_registro).pack(pady=5)

        self.modification_frame = ttk.LabelFrame(self, text="Modificar Campo")
        self.modification_frame.pack(pady=10, padx=10, fill=tk.BOTH, expand=True)

        ttk.Label(self.modification_frame, text="Campo a modificar:").pack(pady=5)
        self.campo_modificar_combo = ttk.Combobox(self.modification_frame, values=self.get_editable_fields())
        self.campo_modificar_combo.pack(pady=5)
        self.campo_modificar_combo.config(state=tk.DISABLED)  # Disable initially

        ttk.Label(self.modification_frame, text="Nuevo valor:").pack(pady=5)
        self.nuevo_valor_entry = ttk.Entry(self.modification_frame)
        self.nuevo_valor_entry.pack(pady=5)
        self.nuevo_valor_entry.config(state=tk.DISABLED)  # Disable initially

        self.modificar_button = ttk.Button(self.modification_frame, text="Modificar Campo", command=self.modificar_campo)
        self.modificar_button.pack(pady=10)
        self.modificar_button.config(state=tk.DISABLED)  # Disable initially

    def get_editable_fields(self):
        return [
            "Paciente - Edad",
            "Paciente - Género",
            "Diagnóstico - Tipo",
            "OD - Dioptría 1",
            "OD - Dioptría 2",
            "OD - Astigmatismo",
            "OD - Fáquico/Pseudo-fáquico",
            "OD - Presión neumática",
            "OD - Presión Perkins",
            "OD - Paquimetría",
            "OD - Longitud axial",
            "OD - Defecto medio (VF_MD)",
            "OS - Dioptría 1",
            "OS - Dioptría 2",
            "OS - Astigmatismo",
            "OS - Fáquico/Pseudo-fáquico",
            "OS - Presión neumática",
            "OS - Presión Perkins",
            "OS - Paquimetría",
            "OS - Longitud axial",
            "OS - Defecto medio (VF_MD)",
        ]

    def seleccionar_registro(self):
        try:
            diagnosis_id_to_edit = int(self.diagnosis_id_entry.get())
            self.record_to_edit = None
            for record in self.records:
                if record.diagnosis_id == diagnosis_id_to_edit:
                    self.record_to_edit = record
                    break

            if self.record_to_edit:
                self.campo_modificar_combo.config(state=tk.NORMAL)
                self.nuevo_valor_entry.config(state=tk.NORMAL)
                self.modificar_button.config(state=tk.NORMAL)
                messagebox.showinfo("Selección", f"Registro con ID {diagnosis_id_to_edit} seleccionado.")
            else:
                self.campo_modificar_combo.config(state=tk.DISABLED)
                self.nuevo_valor_entry.config(state=tk.DISABLED)
                self.modificar_button.config(state=tk.DISABLED)
                messagebox.showerror("Error", f"No se encontró ningún registro con el ID {diagnosis_id_to_edit}.")
        except ValueError:
            self.campo_modificar_combo.config(state=tk.DISABLED)
            self.nuevo_valor_entry.config(state=tk.DISABLED)
            self.modificar_button.config(state=tk.DISABLED)
            messagebox.showerror("Error", "Ingrese un ID de diagnóstico válido.")

    def modificar_campo(self):
        if not self.record_to_edit:
            messagebox.showerror("Error", "Por favor, seleccione un registro primero.")
            return

        campo_seleccionado = self.campo_modificar_combo.get()
        nuevo_valor_str = self.nuevo_valor_entry.get().strip()

        if not campo_seleccionado:
            messagebox.showerror("Error", "Por favor, seleccione el campo que desea modificar.")
            return

        if not nuevo_valor_str:
            messagebox.showerror("Error", "Por favor, ingrese el nuevo valor.")
            return

        try:
            if campo_seleccionado == "Paciente - Edad":
                self.record_to_edit.patient.update_age(int(nuevo_valor_str))
            elif campo_seleccionado == "Paciente - Género":
                self.record_to_edit.patient.update_gender(int(nuevo_valor_str))
            elif campo_seleccionado == "Diagnóstico - Tipo":
                self.record_to_edit.update_diagnosis(int(nuevo_valor_str))
            elif campo_seleccionado == "OD - Dioptría 1":
                self.record_to_edit.eye_examination_od.update_dioptre_1(float(nuevo_valor_str))
            elif campo_seleccionado == "OD - Dioptría 2":
                self.record_to_edit.eye_examination_od.update_dioptre_2(float(nuevo_valor_str))
            elif campo_seleccionado == "OD - Astigmatismo":
                self.record_to_edit.eye_examination_od.update_astigmatism(float(nuevo_valor_str))
            elif campo_seleccionado == "OD - Fáquico/Pseudo-fáquico":
                self.record_to_edit.eye_examination_od.update_phakic_pseudophakic(int(nuevo_valor_str))
            elif campo_seleccionado == "OD - Presión neumática":
                self.record_to_edit.eye_examination_od.update_pneumatic(float(nuevo_valor_str))
            elif campo_seleccionado == "OD - Presión Perkins":
                self.record_to_edit.eye_examination_od.update_perkins(float(nuevo_valor_str))
            elif campo_seleccionado == "OD - Paquimetría":
                self.record_to_edit.eye_examination_od.update_pachymetry(float(nuevo_valor_str))
            elif campo_seleccionado == "OD - Longitud axial":
                self.record_to_edit.eye_examination_od.update_axial_length(float(nuevo_valor_str))
            elif campo_seleccionado == "OD - Defecto medio (VF_MD)":
                self.record_to_edit.eye_examination_od.update_vf_md(float(nuevo_valor_str))
            elif campo_seleccionado == "OS - Dioptría 1":
                self.record_to_edit.eye_examination_os.update_dioptre_1(float(nuevo_valor_str))
            elif campo_seleccionado == "OS - Dioptría 2":
                self.record_to_edit.eye_examination_os.update_dioptre_2(float(nuevo_valor_str))
            elif campo_seleccionado == "OS - Astigmatismo":
                self.record_to_edit.eye_examination_os.update_astigmatism(float(nuevo_valor_str))
            elif campo_seleccionado == "OS - Fáquico/Pseudo-fáquico":
                self.record_to_edit.eye_examination_os.update_phakic_pseudophakic(int(nuevo_valor_str))
            elif campo_seleccionado == "OS - Presión neumática":
                self.record_to_edit.eye_examination_os.update_pneumatic(float(nuevo_valor_str))
            elif campo_seleccionado == "OS - Presión Perkins":
                self.record_to_edit.eye_examination_os.update_perkins(float(nuevo_valor_str))
            elif campo_seleccionado == "OS - Paquimetría":
                self.record_to_edit.eye_examination_os.update_pachymetry(float(nuevo_valor_str))
            elif campo_seleccionado == "OS - Longitud axial":
                self.record_to_edit.eye_examination_os.update_axial_length(float(nuevo_valor_str))
            elif campo_seleccionado == "OS - Defecto medio (VF_MD)":
                self.record_to_edit.eye_examination_os.update_vf_md(float(nuevo_valor_str))

            messagebox.showinfo("Éxito", f"Campo '{campo_seleccionado}' modificado a '{nuevo_valor_str}'.")
            self.callback() # Refresh the main list if needed
            self.destroy()

        except ValueError:
            messagebox.showerror("Error", "Ingrese un valor válido para el campo seleccionado.")
        except Exception as e:
            messagebox.showerror("Error", f"Ocurrió un error al modificar el campo: {e}")

if __name__ == '__main__':
    root = tk.Tk()
    app = EditRegistrosVentana(root, get_diadnosis_records(), lambda: None)
    root.mainloop()